In [ ]:
from __future__ import print_function
from builtins import super
"""
Implementation of Block Neural Autoregressive Flow
http://arxiv.org/abs/1904.04676
"""
import torch.nn.functional as F
import random
import torch
torch.set_printoptions(profile="full")
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as D
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import warnings
import torchvision
print ("torch",torch.__version__)
print ("torchvision",torchvision.__version__)
warnings.filterwarnings("ignore")
import os
import pandas as pd
import sys
import ROOT
import array
import math
import os
import time
import argparse
import pprint
import scipy
from scipy import special
from functools import partial
import json
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tqdm import tqdm
np.set_printoptions(threshold=sys.maxsize)
from sklearn.manifold import TSNE
import csv
from torch.utils import data
from bnaf import *
from tqdm import trange
from datetime import datetime
from dateutil.parser import parse


parser = argparse.ArgumentParser()
parser.add_argument('--cuda', type=int, default=0, help='Which GPU to run on.')
parser.add_argument('--n_steps', type=int, default=100, help='Number of steps to train.')
parser.add_argument('--batch_size', type=int, default=200, help='Training batch size.')
args = parser.parse_args()


# --------------------
# Data
# --------------------
matplotlib.use('tkagg')
class sample:
    def __init__(self, nchan ,biasmag, batch_size, topred, seed=None):
        self.batch_size=batch_size
        self.nchan=nchan
        self.seed=seed
        self.topred=topred
    def sreturn(self):
        xvals=[]
        yvals=[]
        ychan=[]
        if (self.seed==None):
                rng = np.random
        else:
                rng = np.random.RandomState(self.seed)
        #print (-1,1,(self.batch_size,self.nchan))
        randonarr=rng.uniform(-1,1,(self.batch_size,self.nchan))*self.biasmag
        #randonarr[:,1]=1.0*randonarr[:,0]+0.0*randonarr[:,1]


        files = os.listdir()
        ntimep = 100+self.topred
        for bb in range(self.batch_size):


                curx=[]
                cury=[]
                prevarrand=np.array([])
                perm=np.arange(4)
                np.random.shuffle(perm)
                #print(perm)
                ychan.append(perm)
                for nn in range(self.nchan):
                        sinfac=rng.uniform(0.2,1.)*0.7
                        sinarr=np.sin(np.arange(ntimep)*sinfac)*5
                        cosfac=rng.uniform(0.2,1.)*0.7
                        cosarr=np.cos(np.arange(ntimep)*cosfac)*5
                        #print(sinfac)
                        #print(cosfac)
                        corrnoise=True
                        arrand = rng.uniform(-1,1,ntimep)
                        if corrnoise:
                                if prevarrand.size==0:
                                        arrand = rng.uniform(-1,1,ntimep)
                                else:
                                        arrand = rng.uniform(-1,1,ntimep)
                                        arrand[nn*5:]=prevarrand[0:-1*nn*5]
                                cs=np.cumsum(arrand)
                        else:
                                arrand = rng.uniform(-1,1,ntimep)
                                cs=np.cumsum(arrand+randonarr[bb][nn])+sinarr+cosarr

                        #print (sinarr) 
                        curx.append(cs[:-self.topred])
                        cury.append(cs[-self.topred:])
                        #curx.append(np.cumsum(arrand+randonarr[bb][0]))
                        prevarrand=arrand
                curx=np.array(curx)
                cury=np.array(cury)
                #print (ychan[-1])
                #print (curx[:,0])

                curx=curx[ychan[-1]]
                #curx=np.transpose(curx,ychan[-1])
                #print (curx[:,0])
                #print ()
                cury=cury[ychan[-1]]
     
                xvals.append((curx))
                yvals.append((cury))
                #print (np.array(xvals).shape)
                #print (np.array(yvals).shape)
                #plt.plot(curx)
                #plt.xlabel("steps")
                #plt.ylabel("position")
                #plt.title("randdwalk")
                #plt.show()
                #sys.exit()
                #input("Press Enter to continue...")
        return np.array(xvals, dtype="float32"),np.array(yvals, dtype="float32"),np.array(ychan, dtype="int64")


class LSTM(nn.Module):
    def __init__(self,nchan=4,ntime=100,n_hidden=64,n_layers=10,npoint=5):
        super(LSTM, self).__init__()
        self.n_hidden=n_hidden
        self.n_layers=n_layers
        self.ntime=ntime+npoint
        self.nchan=nchan
        self.npoint=npoint 
        self.output_size=self.nchan*self.npoint
        self.ks=10





        self.lin = nn.Sequential(  
        nn.Linear(464, 2*400)
        )

        self.out = nn.Linear(self.n_hidden*self.npoint, self.output_size)


        self.lstm = nn.LSTM(input_size=nchan,hidden_size= self.n_hidden, num_layers=5,batch_first=True)
        self.lstm1 = nn.LSTM(input_size=1,hidden_size= self.n_hidden, num_layers=5,batch_first=True)
        #self.soft = nn.Sequential( nn.Softmax(dim=1))


    def forward(self, inputs):
        inputlstm=inputs[0]
        inputlstmshape=inputlstm.shape
        batchs =inputlstmshape[0]
        inputconv=inputs[1].unsqueeze(1).transpose(1, 2)
        h0c = torch.zeros( 5, batchs, self.n_hidden).to('cuda:'+str(args.cuda))  
        c0c = torch.zeros( 5, batchs, self.n_hidden).to('cuda:'+str(args.cuda))
        LSc,hiddenc=self.lstm1(inputconv, (h0c,c0c)) 


        inputlstm=inputlstm.transpose(1, 2)
        #print(LSc.shape)
        #print(inputlstm.shape)
        h0 = torch.zeros( 5, batchs, self.n_hidden).to('cuda:'+str(args.cuda))  
        c0 = torch.zeros( 5, batchs, self.n_hidden).to('cuda:'+str(args.cuda))
        LS,hidden=self.lstm(inputlstm, hiddenc) 
        
        #output = self.out(torch.cat((LS[:,-1,:],LSc[:,-1,:]),dim=1))
        LS=LS[:,-self.npoint:,:].reshape(LS.size(0), -1)
        output = self.out(LS)
        #print (output.shape)
        return output.reshape(output.shape[0],self.nchan,self.npoint)


class Conv(nn.Module):
    def __init__(self,nchan=4,ntime=100,n_hidden=64,n_layers=10,npoint=5):
        super(Conv, self).__init__()
        self.n_hidden=n_hidden
        self.n_layers=n_layers
        self.ntime=ntime+npoint
        self.nchan=nchan
        self.npoint=npoint 
        self.output_size=self.nchan*self.npoint
        self.ks=10

        self.conv1 = nn.Sequential( 
            nn.Conv1d(self.nchan, self.n_hidden,kernel_size=10, padding=1),
            nn.MaxPool1d(2)
        )
        self.conv2 = nn.Sequential( 
            nn.Conv1d(self.n_hidden, self.n_hidden,stride=1, kernel_size=30, padding=1),
            nn.MaxPool1d(2)
        )

        self.lin = nn.Sequential(  
        nn.Linear(self.n_hidden*9, self.n_hidden)
        )

        self.out = nn.Linear(self.n_hidden, self.nchan*self.nchan)



    def forward(self, inputs):
        batchs =inputs.shape[0]
        CO = F.relu(self.conv1(inputs))      
        CO = F.relu(self.conv2(CO))  
        lin = F.relu(self.lin(CO.reshape(CO.size(0), -1)))

        output = F.softmax(self.out(lin))

        return output.reshape(inputs.shape[0],self.nchan,self.nchan),lin

hdfpath="/cms/knash/sto/"
hdflist=os.listdir(hdfpath)
megaframe=pd.DataFrame()

for ihd,hd in enumerate(hdflist):
        hdstr=hd.replace(".json","")
        stockframe=pd.read_json(hdfpath+hd)
        stockframe.index=stockframe["time"]
        stockframe.index=pd.DatetimeIndex(stockframe.index)

        stockframe = stockframe.drop("time",axis=1)
        stockframe = stockframe.sort_index(axis=0)

        stockframe.columns=[[hdstr,hdstr,hdstr,hdstr,hdstr],['close', 'high', 'low', 'open', 'volume']]

        #print (stockframe)
        if ihd==0:
                megaframe = stockframe.copy()
                prestr=hdstr
        else:
                megaframe = pd.concat( [megaframe,stockframe],axis=1)
                print (megaframe.shape)
                prestr=hdstr

#print (megaframe)

megaframe=megaframe.dropna(axis=0)
megaframe = megaframe.sort_index(axis=0)
#days = (megaframe.index.to_period("D")).copy()
#days=days.drop_duplicates()
startind=0
lenth=30
for im,mf in enumerate(megaframe.index):
        if (im<startind):
                continue
        returnframe=megaframe.iloc[im:im+lenth]
        print (returnframe)

        if (returnframe.shape[0]!=lenth):
                continue

        highs=(returnframe.iloc[:, returnframe.columns.get_level_values(1)=='high'])
        lows=(returnframe.iloc[:, returnframe.columns.get_level_values(1)=='low'])
        vol=(returnframe.iloc[:, returnframe.columns.get_level_values(1)=='volume'])
        highs.columns=highs.columns.droplevel(level=0)
        lows.columns=lows.columns.droplevel(level=0)
        vol.columns=vol.columns.droplevel(level=0)
        highs=highs.values
        lows=lows.values

        mean= 0.5*(highs+lows)
        nrange=np.max(highs)-np.min(lows)
        highs=(highs-mean)/nrange
        lows=(lows-mean)/nrange

        randonorm=np.random.normal(size=highs.shape)

        toreturn=(highs+lows)/2+(100*randonorm*(highs-lows)/2)
        print(toreturn)
        #if (day.dayofweek)!=0:
        #        continue
        #curtrainframe=megaframe.loc[str(day):str(day+4)]
        #strip17=curtrainframe.index[curtrainframe.index.hour==17]
        #curtrainframe=curtrainframe.drop(strip17)
        #print(curtrainframe)


        
    
sys.exit()
nchantot=4
trsamp={}
input_size=200
ntime=100
nepoch=args.n_steps/50
biasvsal=0.4
topred=30
trsamp["train"]=sample(nchantot,biasvsal,input_size,topred)
trsamp["val"]=sample(nchantot,biasvsal,input_size,topred,12529)


modellstm = LSTM(nchan=nchantot,npoint=topred).to('cuda:'+str(args.cuda))
modellstm.train()
modelconv = Conv(nchan=nchantot,npoint=topred).to('cuda:'+str(args.cuda))
modelconv.train()


criterion = nn.MSELoss()

optimizerlstm = torch.optim.Adam(modellstm.parameters(), lr=0.005, weight_decay=0.0)
optimizerconv = torch.optim.Adam(modelconv.parameters(), lr=0.001, weight_decay=0.0)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizerlstm, factor=0.5, patience=0, cooldown=4, verbose=True)
fullval=trsamp["val"].sreturn()
valsamp=torch.from_numpy(fullval[0]).to('cuda:'+str(args.cuda))
valsamptrue=torch.from_numpy(fullval[1]).to('cuda:'+str(args.cuda))
fig, axs = plt.subplots(4)
with tqdm(total=args.n_steps, desc='Start step {}; Training for {} steps'.format(0, args.n_steps)) as pbar:
        for istep in range(args.n_steps):
                curtuple = trsamp["train"].sreturn()
                curdata = torch.from_numpy(curtuple[0]).to('cuda:'+str(args.cuda))
                #print (curdata.shape)

                outputsconv  = modelconv(curdata)
                predchantrue  = outputsconv[0]
                topass  = outputsconv[1]
                outputs  = modellstm((curdata,topass))
                #print (outputs[0],curtuple[1])
                optimizerlstm.zero_grad()
                optimizerconv.zero_grad()
                curtrue = torch.from_numpy(curtuple[1]).to('cuda:'+str(args.cuda))
                #print (curtuple[2])
        
                curchantrue = torch.from_numpy(curtuple[2])
                curchantrue=F.one_hot(curchantrue).to('cuda:'+str(args.cuda)).float()


                curpred = outputs
                #print (curpred.shape )
                #print (curtrue.shape )
                #  curpred=curpred.reshape(curtrue.shape[0],curtrue.shape[1],) 

                #print (torch.std(curtrue-curpred))
                #print (curtrue[0],curpred[0])
                
                loss = criterion(curpred,curtrue).mean()

                lossconv = nn.BCELoss()(predchantrue,curchantrue).mean()

                pbar.set_postfix(L='{:.4f}'.format(loss*1000),Lchan='{:.4f}'.format(lossconv*1000))
                pbar.update()
                lossconv.backward(retain_graph=True)
                optimizerconv.step()
                loss.backward(retain_graph=False)
                optimizerlstm.step()

                if (istep%nepoch==0):

                        for param_group in optimizerlstm.param_groups:
                                print(param_group["lr"])
                        for param_group in optimizerconv.param_groups:
                                print(param_group["lr"])

                        outputsconv  = modelconv(valsamp)
                        predchantrue  = outputsconv[0]
                        topass  = outputsconv[1]
                        outputs  = modellstm((valsamp,topass))

                        #outputs= outputs.reshape(valsamptrue.shape[0],valsamptrue.shape[1])

                        #print (outputs)
                        #print (valsamptrue) 
                        #print (np.array(xvals).shape)
                        #print (np.array(yvals).shape)
                        plot=True
                        if plot:

                                #plt.clf()

                                axs[0].cla()
                                axs[1].cla()
                                axs[2].cla()
                                axs[3].cla()
                                #plt.plot(np.append(valsamp.cpu().numpy()[0,0,:],outputs.cpu().detach().numpy()[0,0,:]))
                                #plt.plot(np.append(valsamp.cpu().numpy()[0,1,:],outputs.cpu().detach().numpy()[0,1,:]))
                                axs[0].plot(np.append(valsamp.cpu().numpy()[0,0,:],outputs.cpu().detach().numpy()[0,0,:]))
                                axs[1].plot(np.append(valsamp.cpu().numpy()[0,1,:],outputs.cpu().detach().numpy()[0,1,:]))
                                axs[2].plot(np.append(valsamp.cpu().numpy()[0,2,:],outputs.cpu().detach().numpy()[0,2,:]))
                                axs[3].plot(np.append(valsamp.cpu().numpy()[0,3,:],outputs.cpu().detach().numpy()[0,3,:]))
                                #plt.xlabel("steps")
                                #plt.ylabel("position")
                                #plt.title("randdwalk")
                                plt.show(block=False)
                                plt.pause(1)

                        #sys.exit()
                        validation_loss=criterion(outputs,valsamptrue).mean()*1000
                        scheduler.step(validation_loss)
                        print (validation_loss.tolist())





